<a href="https://colab.research.google.com/github/farahbakhsh3/X1/blob/master/atari-rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.dropbox.com/s/rbgg6mbunq8qtr3/newCallback.py
#!wget ????
!wget https://www.dropbox.com/s/g69yj8ia900bk1p/dqn_Atari_512.py

#!pip install tensorflow==1.6.0
!apt-get install cmake
#!apt-get install swig
!pip install gym[atari]
#!pip install gym[Box2D]
#!pip install keras-rl

!pip install dropbox

!git clone https://github.com/matthiasplappert/keras-rl.git
#import os
#os.chdir('keras-rl/')
%cd keras-rl/
!python setup.py install
#!pip install -e .
#os.chdir('../')
import rl
%cd ../


!python3 ./dqn_Atari_512.py

--2019-02-03 07:50:16--  https://www.dropbox.com/s/rbgg6mbunq8qtr3/newCallback.py
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/rbgg6mbunq8qtr3/newCallback.py [following]
--2019-02-03 07:50:16--  https://www.dropbox.com/s/raw/rbgg6mbunq8qtr3/newCallback.py
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf28e9a5d25accd7d6825659c78.dl.dropboxusercontent.com/cd/0/inline/AalVIS5P6E5wWA-2ZFWZ6kIoMmIW2EkWS2wBTMvNsf_cIJg6-E2Ey5BSlOQivLpTzrZALUJ0yek-mCdhVPaZdItTMvj8375b8makAM22snkIXQ/file# [following]
--2019-02-03 07:50:17--  https://ucf28e9a5d25accd7d6825659c78.dl.dropboxusercontent.com/cd/0/inline/AalVIS5P6E5wWA-2ZFWZ6kIoMmIW2EkWS2wBTMvNsf_cIJg6-E2Ey5BSlOQivLpTzrZALUJ0yek-mCdhVPaZdItTMvj8375b8makAM22snkIXQ/file
Resolving ucf

In [9]:
from PIL import Image
import numpy as np
import gym
   
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Permute
from keras.layers import MaxPooling2D, BatchNormalization, Dropout
from keras.optimizers import Adam
   
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory, EpisodeParameterMemory
from rl.core import Processor
from rl.callbacks import FileLogger#, ModelIntervalCheckpoint
from newCallback import ModelIntervalCheckpoint
   
   
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4
   
   
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L') 
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  
   
    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch
   
    # def process_reward(self, reward):
    #     return np.clip(reward, -1., 1.)
    #     return reward
   
    # def process_action(self, action):        
    #     return action
   
   
class args():
    pass
   
   
args.mode = 'train'
args.env_name = 'Boxing-v4'
args.weights = None
   
print('========================')
print(args.mode)
print(args.env_name)
print(args.weights)
print('========================')
   
   
env = gym.make(args.env_name)
np.random.seed(7)
env.seed(7)
nb_actions = env.action_space.n
   
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
# (width, height, channels)
model.add(Permute((2, 3, 1), input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(2, 2), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(128, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))
print(model.summary())
   
# memory = EpisodeParameterMemory(limit=1000000, window_length=WINDOW_LENGTH)
memory = SequentialMemory(limit=10000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
   
# policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
#                              nb_steps=1000000)
policy = BoltzmannQPolicy()
# policy = EpsGreedyQPolicy(eps=.1)
   
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=1000, gamma=.99, target_model_update=1000,
               train_interval=1, delta_clip=1.)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
   
weights_filename = 'dqn512_{}_weights.h5f'.format(args.env_name)
checkpoint_weights_filename = 'dqn512_' + args.env_name + '_weights_{step}.h5f'
log_filename = 'dqn512_{}_log.json'.format(args.env_name)
if args.mode == 'train':
       
    callbacks = None
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=50001, verbose=1, updropbox=True)]
    # callbacks += [FileLogger(log_filename, interval=100)]
    
    if (args.weights):
      dqn.load_weights(weights_filename) 
      print('Weights Loaded: ', weights_filename)
      
    dqn.fit(env, callbacks=callbacks, nb_steps=1000000, verbose=2, log_interval=100000, visualize=False)
   
    dqn.save_weights(weights_filename, overwrite=True)
    print('Weights Saved: ', weights_filename)
   
    dqn.test(env, nb_episodes=10, visualize=False)
elif args.mode == 'test':
    if args.weights:
        weights_filename = args.weights
    dqn.load_weights(weights_filename)
    print('Weights Loaded: ', weights_filename)
    dqn.test(env, nb_episodes=10, visualize=True)

train
Boxing-v4
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_3 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 84, 84, 32)        1184      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 42, 42, 32)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 42, 42, 32)        128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 42, 42, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 21, 21, 64)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 21, 21, 64)        

KeyboardInterrupt: ignored